# Análisis de esfuerzo logístico en torneos de Pelota a Paleta

## Contexto
En la organización de Torneos en la cual participo surgió un pedido recurrente de bonificaciones
para jugadores que residen lejos de la sede organizativa, bajo el supuesto de que
incurren en mayores costos de traslado y viáticos.

## Hipótesis inicial
La hipótesis dominante era que la distancia geográfica entre el lugar de origen del
jugador y la sede de competencia era un buen proxy del esfuerzo logístico total.

## Enfoque
En lugar de tomar una decisión basada en percepciones, se decidió analizar el esfuerzo
logístico real a partir de datos objetivos: distancia recorrida y frecuencia de
participación en eventos.

In [24]:
import pandas as pd
import numpy as np
from math import radians, sin, cos, sqrt, atan2
import plotly.express as px
pd.set_option("display.max_columns", None)

In [25]:
df = pd.read_csv('./data/jugadores.csv')
df.head()

,jugador_id,torneo_id,asistencia,origen,origen_lat,origen_lon,destino,destino_lat,destino_lon
0,5,110,1,Venado Tuerto,-33.745557,-61.969016,16,-33.745947,-61.963890
1,5,112,1,Venado Tuerto,-33.745557,-61.969016,6,-33.700647,-61.613670
2,5,114,1,Venado Tuerto,-33.745557,-61.969016,20,-33.324484,-62.041291
3,5,116,1,Venado Tuerto,-33.745557,-61.969016,1,-33.565943,-62.884720
4,5,118,1,Venado Tuerto,-33.745557,-61.969016,16,-33.745947,-61.963890


In [26]:
df.info()
df.isna().sum()

<class 'pandas.DataFrame'>
RangeIndex: 1204 entries, 0 to 1203
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   jugador_id   1204 non-null   int64  
 1   torneo_id    1204 non-null   int64  
 2   asistencia   1204 non-null   int64  
 3   origen       1204 non-null   str    
 4   origen_lat   1204 non-null   float64
 5   origen_lon   1204 non-null   float64
 6   destino      1204 non-null   int64  
 7   destino_lat  1204 non-null   float64
 8   destino_lon  1204 non-null   float64
dtypes: float64(4), int64(4), str(1)
memory usage: 84.8 KB


jugador_id     0
torneo_id      0
asistencia     0
origen         0
origen_lat     0
origen_lon     0
destino        0
destino_lat    0
destino_lon    0
dtype: int64

In [27]:
def haversine(lat1, lon1, lat2, lon2):
    R = 6371  # Radio de la Tierra en km

    lat1, lon1, lat2, lon2 = map(
        radians, [lat1, lon1, lat2, lon2]
    )

    dlat = lat2 - lat1
    dlon = lon2 - lon1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    return R * c

In [28]:
df["distancia_km"] = df.apply(
    lambda row: haversine(
        row["origen_lat"],
        row["origen_lon"],
        row["destino_lat"],
        row["destino_lon"]
    ),
    axis=1
)

df[["jugador_id", "torneo_id", "distancia_km"]].head()

,jugador_id,torneo_id,distancia_km
0,5,110,0.475993
1,5,112,33.241135
2,5,114,47.297918
3,5,116,87.075574
4,5,118,0.475993


In [29]:
jugador_metrics = (
    df.groupby(["jugador_id"], as_index=False)
      .agg(
          km_totales=("distancia_km", "sum"),
          km_prom=("distancia_km", "mean"),
          cantidad_eventos=("torneo_id", "nunique")
      )
)

jugador_metrics.head()

,jugador_id,km_totales,km_prom,cantidad_eventos
0,5,241.866338,34.552334,7
1,9,249.959949,83.319983,3
2,10,167.328037,41.832009,4
3,11,3.419984,3.419984,1
4,17,2259.276033,141.204752,16


In [30]:
jugador_stats = (
    df.groupby("jugador_id")
      .agg(
          localidad=("origen", "max"),
          origen_lat=("origen_lat", "mean"),
          origen_lon=("origen_lon", "mean"),
          km_totales=("distancia_km", "sum"),
          km_prom=("distancia_km", "mean"),
          km_min=("distancia_km", "min"),
          km_max=("distancia_km", "max"),
          km_std=("distancia_km", "std"),
          km_p90=("distancia_km", lambda x: x.quantile(0.90)),
          cantidad_eventos=("torneo_id", "nunique"),
          sedes_visitadas=("destino", "nunique")
      )
      .reset_index()
)
jugador_stats.head()

,jugador_id,localidad,origen_lat,origen_lon,km_totales,km_prom,km_min,km_max,km_std,km_p90,cantidad_eventos,sedes_visitadas
0,5,Venado Tuerto,-33.745557,-61.969016,241.866338,34.552334,0.475993,87.075574,30.259808,63.391213,7,6
1,9,Paraná,-31.743465,-60.508353,249.959949,83.319983,29.966153,109.996898,46.205772,109.996898,3,2
2,10,Paraná,-31.743465,-60.508353,167.328037,41.832009,3.419984,109.996898,46.778030,85.084502,4,3
3,11,Paraná,-31.743465,-60.508353,3.419984,3.419984,3.419984,3.419984,NaN,3.419984,1,1
4,17,Rosario,-32.958702,-60.693042,2259.276033,141.204752,75.877775,214.667025,47.748116,211.741828,16,10


In [31]:
sistema_stats = jugador_stats.describe(percentiles=[0.5, 0.75, 0.9])
sistema_stats

,jugador_id,origen_lat,origen_lon,km_totales,km_prom,km_min,km_max,km_std,km_p90,cantidad_eventos,sedes_visitadas
count,340.000000,340.000000,340.000000,340.000000,340.000000,340.000000,340.000000,205.000000,340.000000,340.000000,340.000000
mean,517.888235,-32.960202,-61.754174,314.095087,77.835569,47.650733,117.420157,49.741527,104.387902,3.541176,2.861765
std,266.750476,1.050293,1.057767,484.799881,93.036418,91.468425,116.137855,38.927064,106.659390,3.587270,2.466593
min,5.000000,-39.005881,-68.359902,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000
50%,589.500000,-33.375129,-61.649694,115.473246,54.723341,7.405592,84.188769,39.189333,77.024208,2.000000,2.000000
75%,756.750000,-31.743465,-61.157817,363.034666,105.501407,73.511080,214.156202,70.857033,162.491716,4.000000,4.000000
90%,820.100000,-31.388253,-60.508353,918.134500,175.201891,105.950956,282.521799,109.670519,257.434496,9.000000,7.000000
max,860.000000,-30.951201,-58.674461,3351.641645,848.550003,848.550003,848.550003,193.719023,848.550003,18.000000,11.000000


In [32]:
jugador_stats["rank_km_totales"] = jugador_stats["km_totales"].rank(pct=True)
jugador_stats["rank_km_prom"] = jugador_stats["km_prom"].rank(pct=True)
jugador_stats.head()

,jugador_id,localidad,origen_lat,origen_lon,km_totales,km_prom,km_min,km_max,km_std,km_p90,cantidad_eventos,sedes_visitadas,rank_km_totales,rank_km_prom
0,5,Venado Tuerto,-33.745557,-61.969016,241.866338,34.552334,0.475993,87.075574,30.259808,63.391213,7,6,0.647059,0.352941
1,9,Paraná,-31.743465,-60.508353,249.959949,83.319983,29.966153,109.996898,46.205772,109.996898,3,2,0.661765,0.661765
2,10,Paraná,-31.743465,-60.508353,167.328037,41.832009,3.419984,109.996898,46.778030,85.084502,4,3,0.567647,0.414706
3,11,Paraná,-31.743465,-60.508353,3.419984,3.419984,3.419984,3.419984,NaN,3.419984,1,1,0.136765,0.142647
4,17,Rosario,-32.958702,-60.693042,2259.276033,141.204752,75.877775,214.667025,47.748116,211.741828,16,10,0.991176,0.841176


In [33]:
## Distribución de pesos para cada metrica

jugador_stats["indice_esfuerzo_base"] = (
    jugador_stats["km_totales"] * 0.35 +
    jugador_stats["km_p90"] * 0.25 +
    jugador_stats["cantidad_eventos"] * 0.40
)

In [34]:
## Acá penalizamos la baja participación a eventos ya que queremos fomentar la recurrencia a los torneos

jugador_stats["participacion_norm"] = (
    jugador_stats["cantidad_eventos"] /
    jugador_stats["cantidad_eventos"].max()
)

jugador_stats["indice_esfuerzo"] = (
    jugador_stats["indice_esfuerzo_base"] *
    (jugador_stats["participacion_norm"] ** 0.5)
)

In [35]:
jugador_stats.sort_values("indice_esfuerzo", ascending=False).head(10)

,jugador_id,localidad,origen_lat,origen_lon,km_totales,km_prom,km_min,km_max,km_std,km_p90,cantidad_eventos,sedes_visitadas,rank_km_totales,rank_km_prom,indice_esfuerzo_base,participacion_norm,indice_esfuerzo
233,733,Rafaela,-31.254750,-61.528983,3351.641645,209.477603,7.405592,286.770967,87.377036,281.296920,16,11,1.000000,0.944118,1249.798806,0.888889,1178.321614
116,395,Rafaela,-31.254750,-61.528983,2914.533953,208.180997,7.405592,286.770967,94.561113,281.818123,14,10,0.997059,0.941176,1096.141414,0.777778,966.705861
121,405,Bell Ville,-32.630508,-62.688857,2595.456588,144.192033,97.996467,226.251864,41.104732,200.363525,18,11,0.994118,0.847059,965.700687,1.000000,965.700687
4,17,Rosario,-32.958702,-60.693042,2259.276033,141.204752,75.877775,214.667025,47.748116,211.741828,16,10,0.991176,0.841176,850.082069,0.888889,801.465060
119,402,Paraná,-31.743465,-60.508353,2224.181971,185.348498,3.419984,300.915648,90.975038,258.969120,12,10,0.988235,0.917647,848.005970,0.666667,692.393975
104,366,Alcorta,-33.538969,-61.124381,1802.237177,106.013952,0.543609,261.876627,78.645723,230.254922,17,10,0.979412,0.761765,695.146743,0.944444,675.561203
48,154,Arroyo Seco,-33.152828,-60.587072,2019.537630,144.252688,90.205987,218.275245,41.759149,205.075120,14,8,0.985294,0.850000,763.706951,0.777778,673.526222
74,246,Paraná,-31.743465,-60.508353,1965.937484,196.593748,3.419984,300.915648,110.091350,300.915648,10,8,0.982353,0.932353,767.307031,0.555556,571.916894
148,526,Rufino,-34.263238,-62.712828,1776.540502,136.656962,79.141424,350.170520,69.763913,162.542101,13,8,0.976471,0.829412,667.624701,0.722222,567.371896
156,546,Rufino,-34.263238,-62.712828,1632.195384,136.016282,79.141424,350.170520,72.826075,164.816724,12,8,0.970588,0.826471,617.272565,0.666667,504.000939


In [36]:
jugador_stats[
    ["km_totales", "km_prom", "km_p90", "cantidad_eventos", "indice_esfuerzo"]
].corr()

,km_totales,km_prom,km_p90,cantidad_eventos,indice_esfuerzo
km_totales,1.000000,0.452192,0.611138,0.817506,0.982395
km_prom,0.452192,1.000000,0.940987,0.115594,0.324337
km_p90,0.611138,0.940987,1.000000,0.337159,0.489489
cantidad_eventos,0.817506,0.115594,0.337159,1.000000,0.849221
indice_esfuerzo,0.982395,0.324337,0.489489,0.849221,1.000000


In [37]:
fig = px.scatter(
    jugador_stats,
    x="km_prom",
    y="km_totales",
    color="indice_esfuerzo",
    hover_data={
        "jugador_id": True,
        "localidad": True,
        "km_prom": ':.1f',
        "km_totales": ':.1f',
        "cantidad_eventos": True,
        "indice_esfuerzo": ':.1f'
    },
    title="Esfuerzo real vs distancia promedio",
    labels={
        "indice_esfuerzo": "Índice de esfuerzo",
        "jugador_id": "Jugador",
        "km_totales": "Km Acumulados",
        "km_prom": "Km prom. por evento",
        "cantidad_eventos": "Cantidad de eventos",
        "localidad": "Localidad"
    },
    template="plotly_dark"
)

fig.update_traces(
    marker=dict(size=12, opacity=0.8)
)

fig.update_layout(
    height=600,
    width=900
)

fig.show()

In [38]:
top_n = 10
df_top = jugador_stats.sort_values("indice_esfuerzo", ascending=False).head(top_n)
df_top["jugador_id"] = df_top["jugador_id"].astype(str)
fig = px.bar(
    df_top,
    x="indice_esfuerzo",
    y="jugador_id",
    orientation="h",
    color="indice_esfuerzo",
    hover_data={
        "localidad": True,
        "km_totales": ':.1f',
        "km_prom": ':.1f',
        "cantidad_eventos": True,
        "indice_esfuerzo":':.1f'
    },
    title="Top jugadores por índice de esfuerzo",
    labels={
        "jugador_id": "Jugador",
        "indice_esfuerzo": "Índice de esfuerzo",
        "km_totales": "Km Acumulados",
        "km_prom": "Km promedio por evento",
        "cantidad_eventos": "Cantidad de eventos",
        "localidad": "Localidad",
        "km_p90":"Km. Perc.90"
    },
    template="plotly_dark"
)

fig.update_layout(
    yaxis=dict(autorange="reversed"),
    height=500,
    width=900
)

fig.show()


In [39]:
fig = px.scatter(
    jugador_stats,
    x="km_prom",
    y="cantidad_eventos",
    size="indice_esfuerzo",
    color="indice_esfuerzo",
    hover_data={
        "jugador_id": True,
        "localidad": True,
        "km_prom":':.1f',
        "indice_esfuerzo":':.1f',
        "km_totales": ':.0f',
        "km_p90": ':.0f'
    },
    title="Frecuencia vs distancia (tamaño = esfuerzo total)",
    labels={
        "km_totales": "Km Acumulados",
        "km_prom": "Km promedio por evento",
        "cantidad_eventos": "Cantidad de eventos",
        "indice_esfuerzo": "Índice de esfuerzo",
        "jugador_id": "Jugador",
        "localidad": "Localidad",
        "km_p90":"Km. Perc.90"
    },
    template="plotly_dark",
    size_max=45
)

fig.update_layout(
    height=600,
    width=900
)

fig.show()


In [40]:
# Percentil del índice de esfuerzo 
# Donde los valores de retorno serán la cantidad de pases libres que recibirá el jugador
 
jugador_stats["percentil_esfuerzo"] = (
    jugador_stats["indice_esfuerzo"].rank(pct=True)
)

def asignar_beneficio(p):
    if p >= 0.97:
        return 3
    elif p >= 0.95:
        return 2
    elif p >= 0.92:
        return 1
    else:
        return 0

jugador_stats["beneficio"] = jugador_stats["percentil_esfuerzo"].apply(asignar_beneficio)

In [41]:
fig = px.scatter(
    jugador_stats,
    x="km_prom",
    y="cantidad_eventos",
    color="beneficio",
    size="indice_esfuerzo",
    hover_data={
        "jugador_id": True,
        "km_totales": ":.1f",
        "km_p90": ':.1f',
        "km_prom": ':.1f',
        "percentil_esfuerzo": ":.2f",
        "indice_esfuerzo":':.1f'
        
    },
    title="Asignación de beneficios basada en esfuerzo acumulado",
    labels={
        "jugador_id": "Jugador",
        "km_totales": "Km Acumulados",
        "km_p90":"Km. Perc.90",
        "km_prom": "Km promedio por evento",
        "indice_esfuerzo": "Índice de Esfuerzo",
        "percentil_esfuerzo": "Perc. de Esfuerzo",
        "cantidad_eventos": "Cantidad de eventos",
        "localidad": "Localidad"
    },
    template="plotly_dark"
)

fig.update_layout(
    xaxis_title="Km promedio por evento",
    yaxis_title="Cantidad de eventos disputados",
)

fig.show()


In [42]:
map_df = jugador_stats.copy()
map_df["origen_lat"] = map_df["origen_lat"].astype(float)
map_df["origen_lon"] = map_df["origen_lon"].astype(float)

In [43]:
map_df.head()

,jugador_id,localidad,origen_lat,origen_lon,km_totales,km_prom,km_min,km_max,km_std,km_p90,cantidad_eventos,sedes_visitadas,rank_km_totales,rank_km_prom,indice_esfuerzo_base,participacion_norm,indice_esfuerzo,percentil_esfuerzo,beneficio
0,5,Venado Tuerto,-33.745557,-61.969016,241.866338,34.552334,0.475993,87.075574,30.259808,63.391213,7,6,0.647059,0.352941,103.301022,0.388889,64.419505,0.726471,0
1,9,Paraná,-31.743465,-60.508353,249.959949,83.319983,29.966153,109.996898,46.205772,109.996898,3,2,0.661765,0.661765,116.185207,0.166667,47.432412,0.670588,0
2,10,Paraná,-31.743465,-60.508353,167.328037,41.832009,3.419984,109.996898,46.778030,85.084502,4,3,0.567647,0.414706,81.435939,0.222222,38.389270,0.605882,0
3,11,Paraná,-31.743465,-60.508353,3.419984,3.419984,3.419984,3.419984,NaN,3.419984,1,1,0.136765,0.142647,2.451991,0.055556,0.577940,0.136765,0
4,17,Rosario,-32.958702,-60.693042,2259.276033,141.204752,75.877775,214.667025,47.748116,211.741828,16,10,0.991176,0.841176,850.082069,0.888889,801.465060,0.991176,3


In [44]:
beneficiados = map_df[map_df["beneficio"]> 0].copy()

# Crear el mapa
fig = px.scatter_mapbox(
    beneficiados,
    lat="origen_lat",
    lon="origen_lon",
    color="beneficio",
    size="indice_esfuerzo",
    color_discrete_map={'1': "#912cf0", '2': "#d77c32", '3': "#d6e13c"},
    size_max=30,
    hover_data={
        "jugador_id": True,
        "localidad": True,
        "indice_esfuerzo": ":.1f",
        "cantidad_eventos": True,
        "km_totales": ":.1f",
        "beneficio": True,
        "origen_lat": False,
        "origen_lon": False
    },
    hover_name="localidad",
    zoom=6,
    height=700,
    title="Distribución geográfica de beneficios por índice de esfuerzo"
)

# Mejores estilos para mapa oscuro
fig.update_layout(
    mapbox_style="carto-darkmatter",
    margin={"r": 0, "t": 80, "l": 0, "b": 0},
    mapbox=dict(
        zoom=6,
        bearing=0,
        pitch=0,
    ),
    title=dict(
        text="<b>Distribución geográfica de beneficios</b><br>"
             "<sup>Tamaño: Índice de esfuerzo | Color: Beneficio</sup>",
        font=dict(size=20, color="white"),
        x=0.5,
        xanchor="center"
    ),
    paper_bgcolor="#1e1e1e",
    plot_bgcolor="#1e1e1e",
    legend=dict(
        title=dict(text="Beneficio", font=dict(color="white")),
        font=dict(color="white"),
        bgcolor="rgba(0,0,0,0.5)"
    )
)

fig.update_mapboxes(
    bearing=0,
    pitch=0,
    zoom=6
)

fig.update_traces(
    hovertemplate="<b>%{hovertext}</b><br>" +
                  "Jugador ID: %{customdata[0]}<br>" +
                  "Localidad: %{customdata[1]}<br>" +
                  "Indice Esfuerzo: %{marker.size:.2f}<br>" +
                  "Eventos: %{customdata[3]}<br>" +
                  "Km totales: %{customdata[4]:.1f}<br>" +
                  "Beneficio: %{customdata[5]}<br>" +
                  "<extra></extra>"
)

fig.show()

/tmp/ipykernel_8438/4283268504.py:4: DeprecationWarning:

*scatter_mapbox* is deprecated! Use *scatter_map* instead. Learn more at: https://plotly.com/python/mapbox-to-maplibre/



In [45]:
# Beneficiados con nuestro metodo
aplicaciones = beneficiados[['jugador_id','localidad','cantidad_eventos', 'km_totales', 'indice_esfuerzo', 'beneficio']].sort_values(['beneficio', 'indice_esfuerzo'], ascending=False)
print(aplicaciones)
print(f"Jugadores beneficiados: {aplicaciones['jugador_id'].nunique()}")
print(f"Beneficios totales: {aplicaciones['beneficio'].sum()}")

     jugador_id                   localidad  cantidad_eventos   km_totales  \
233         733                     Rafaela                16  3351.641645   
116         395                     Rafaela                14  2914.533953   
121         405                  Bell Ville                18  2595.456588   
4            17                     Rosario                16  2259.276033   
119         402                      Paraná                12  2224.181971   
104         366                     Alcorta                17  1802.237177   
48          154                 Arroyo Seco                14  2019.537630   
74          246                      Paraná                10  1965.937484   
148         526                      Rufino                13  1776.540502   
156         546                      Rufino                12  1632.195384   
9            25                      Paraná                10  1685.360370   
134         462                   Esperanza                10  1

## Hallazgos clave

- La distancia lineal al centro organizativo no explica el esfuerzo total.
- Jugadores de zonas céntricas acumulan más kilómetros debido a una mayor
  participación en eventos.
- El esfuerzo logístico es una combinación de distancia y frecuencia participativa.

## Impacto organizacional

A partir de este análisis, el criterio de bonificación fue redefinido para considerar
el esfuerzo logístico real, logrando una asignación más equitativa de los beneficios.

## Conclusión

Los datos permitieron cuestionar un supuesto ampliamente aceptado y mejorar la calidad
de la toma de decisiones dentro de la organización en cuanto aplicación de bonificaciones o pases libres.